##importing dataset



In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pwd

/content


###01 - classic nlp solution

In [3]:
import numpy as np 
import pandas as pd 

In [4]:
train =pd.read_csv('drive/My Drive/Jigsaw/train.csv')
test = pd.read_csv('drive/My Drive/Jigsaw/test.csv')
df_test = test['comment_text']
df = train[['target','comment_text']]

In [ ]:
df.sample(5)

,target,comment_text
869644,0.0,"Well, I am withdrawing from NAFTA. When I shop..."
1566918,0.0,"Dissent, in and of itself, isn't disrespectful..."
681318,0.0,"Thank you very much Thomas, means a lot."
588971,0.0,Tell that to EFC while you are at it.
1140162,0.0,The Crosstrek is the younger little brother of...


In [ ]:
df.shape

(1804874, 2)

In [ ]:
X = df['comment_text']
X.sample(5)

1117274    Your comment isn't sound. While colluding or o...
325897     The most important question has not been answe...
1216656    Enough is enough with this liberal BS. When my...
1554095    No, Dave, that's just not how it works.  The S...
1210503    The media should have reported that it was in ...
Name: comment_text, dtype: object

In [ ]:
df = pd.DataFrame(train,columns=['target','comment_text'])

In [ ]:
y = df['target']
y = np.where(y>=0.5,1.,0.)
 

In [ ]:
df.loc[df["target"]>=0.5 , "target" ] = 1.
df.loc[df["target"]<0.5 , "target"] = 0.
#df['target'] =  np.where(df['target']>=0.5,1.,0.)
#y[:5]
print(df[:5])

   target                                       comment_text
0     0.0  This is so cool. It's like, 'would you want yo...
1     0.0  Thank you!! This would make my life a lot less...
2     0.0  This is such an urgent design problem; kudos t...
3     0.0  Is this something I'll be able to install on m...
4     1.0               haha you guys are a bunch of losers.


In [ ]:
unique, counts = np.unique(y, return_counts=True)
unique, counts

(array([0., 1.]), array([1660540,  144334]))

####Baseline model (using pipeline)

In [ ]:
from imblearn.under_sampling import RandomUnderSampler
X = df['comment_text']
X = X.values.reshape(-1,1)
# comments in one column

y = df['target']
y = np.where(y>=0.5,1.,0.)
undersample = RandomUnderSampler(sampling_strategy='majority')
X_sample, y_sample = undersample.fit_resample(X, y)

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X_sample.reshape(-1), y_sample, test_size=0.3, random_state=42)

In [ ]:
unique, counts = np.unique(y_train, return_counts=True)
unique, counts

(array([0., 1.]), array([101205, 100862]))

In [ ]:
neg_class_len = np.sum(y_train)
#toxic comments
pos_class_len =y_train.shape[0] - int(neg_class_len)
#non - toxic comments
print('negitive (1) class len is :',int(neg_class_len))
print('positive (0) class len is :',pos_class_len)

negitive (1) class len is : 100862
positive (0) class len is : 101205


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

vec = CountVectorizer()
clf = LogisticRegression()
pipe = make_pipeline(vec, clf)
pipe.fit(x_train, y_train);

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [ ]:
from sklearn import metrics

def print_report(pipe, x_test, y_test):
    y_pred = pipe.predict(x_test)
    report = metrics.classification_report(y_test, y_pred)
    print(report)
    print("accuracy: {:0.3f}".format(metrics.accuracy_score(y_test, y_pred)))

print_report(pipe, x_test, y_test)

              precision    recall  f1-score   support

         0.0       0.84      0.90      0.87     43129
         1.0       0.89      0.83      0.86     43472

    accuracy                           0.86     86601
   macro avg       0.86      0.86      0.86     86601
weighted avg       0.86      0.86      0.86     86601

accuracy: 0.863


###a look inside the model

In [ ]:
!pip install eli5

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 216 kB 5.2 MB/s 
     |████████████████████████████████| 133 kB 44.3 MB/s 
  Created wheel for eli5: filename=eli5-0.13.0-py2.py3-none-any.whl size=107748 sha256=087028ad1472bbf2d97c316276de615ab38dd1123ece120c218b694be76fbd4c
  Stored in directory: /root/.cache/pip/wheels/cc/3c/96/3ead31a8e6c20fc0f1a707fde2e05d49a80b1b4b30096573be
Successfully built eli5
  Attempting uninstall: jinja2
    Found existing installation: Jinja2 2.11.3
    Uninstalling Jinja2-2.11.3:
      Successfully uninstalled Jinja2-2.11.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
flask 1.1.4 requires Jinja2<3.0,>=2.10.1, but you have jinja2 3.1.2 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is inco

In [ ]:
import eli5
eli5.show_weights(clf, vec=vec, top=20)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Weight?,Feature
+6.912,idiotic
+6.717,idiots
+6.147,stupidity
+6.075,idiot
+5.998,morons
+5.915,shit
+5.914,moron
+5.713,hypocrite
+5.577,stupid
+4.878,scum


In [ ]:
for _, row in df.sample(5).iterrows():
    print(f"true label: {row['target']}")
    display(eli5.show_prediction(clf, row['comment_text'], vec=vec,))
    print("--"*50)

true label: 0.0


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


----------------------------------------------------------------------------------------------------
true label: 0.0


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


----------------------------------------------------------------------------------------------------
true label: 0.0


...

----------------------------------------------------------------------------------------------------
true label: 0.0


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


----------------------------------------------------------------------------------------------------
true label: 0.0


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


----------------------------------------------------------------------------------------------------


##Try TFidf with some processing

In [ ]:
#vec = TfidfVectorizer(analyzer='char_wb', ngram_range=(3, 5), min_df=.01, max_df=.5)
#clf = LinearSVC()
#pipe_tfidf = make_pipeline(vec, clf)
#pipe_tfidf.fit(x_train, y_train)

In [ ]:
#print_report(pipe_tfidf, x_test, y_test)

In [ ]:
#eli5.show_weights(clf, vec=vec, top=20)

In [ ]:
#for _, row in df.sample(5).iterrows():
#  print(f"true label: {row['target']}")
#   display(eli5.show_prediction(clf, row['comment_text'], vec=vec,))
#   print("--"*50)

###02 - classic nlp solution


In [ ]:
m = df['comment_text']
m.shape

(1804874,)

In [ ]:
from imblearn.under_sampling import RandomUnderSampler
X = df['comment_text']
X = X.values.reshape(-1,1)
# comments in one column

y = df['target']
y = np.where(y>=0.5,1.,0.)
undersample = RandomUnderSampler(sampling_strategy='majority')
X_sample, y_sample = undersample.fit_resample(X, y)

In [ ]:
unique, counts = np.unique(y_sample, return_counts=True)
unique, counts

(array([0., 1.]), array([144334, 144334]))

In [ ]:
#ifrom sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X_sample.reshape(-1), y_sample, 
                                                    test_size=0.3, random_state=42)

In [ ]:
x_train.shape

(202067,)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline

vec = CountVectorizer()
clf = LogisticRegression()
pipe = make_pipeline(vec, clf)
pipe.fit(x_train, y_train);

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [ ]:
from sklearn import metrics

def print_report(pipe, x_test, y_test):
    y_pred = pipe.predict(x_test)
    report = metrics.classification_report(y_test, y_pred)
    print(report)
    print("accuracy: {:0.3f}".format(metrics.accuracy_score(y_test, y_pred)))

print_report(pipe, x_test, y_test)

              precision    recall  f1-score   support

         0.0       0.84      0.90      0.87     43129
         1.0       0.89      0.83      0.86     43472

    accuracy                           0.87     86601
   macro avg       0.87      0.87      0.86     86601
weighted avg       0.87      0.87      0.86     86601

accuracy: 0.865


In [ ]:
import eli5
eli5.show_weights(clf, vec=vec, top=20)

ModuleNotFoundError: ignored

In [ ]:
for _, row in df.sample(5).iterrows():
    print(f"true label: {row['target']}")
    display(eli5.show_prediction(clf, row['comment_text'], vec=vec,))
    print("--"*50)

true label: 0.0


NameError: ignored

In [ ]:
vec2 = TfidfVectorizer(analyzer='char_wb', ngram_range=(3, 5), min_df=.01, max_df=.5)
clf2 = LinearSVC()
pipe_tfidf = make_pipeline(vec2, clf2)
pipe_tfidf.fit(x_train, y_train)

Pipeline(steps=[('tfidfvectorizer',
                 TfidfVectorizer(analyzer='char_wb', max_df=0.5, min_df=0.01,
                                 ngram_range=(3, 5))),
                ('linearsvc', LinearSVC())])

In [ ]:
print_report(pipe_tfidf, x_test, y_test)

              precision    recall  f1-score   support

         0.0       0.83      0.88      0.85     43129
         1.0       0.87      0.82      0.84     43472

    accuracy                           0.85     86601
   macro avg       0.85      0.85      0.85     86601
weighted avg       0.85      0.85      0.85     86601

accuracy: 0.848


In [ ]:
eli5.show_weights(clf2, vec=vec2, top=20)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Weight?,Feature
+9.697,crap
+9.183,hit
+9.044,damn
+8.437,suck
+8.086,moro
+7.834,fool
+7.671,fool
+6.825,moron
+6.645,liar
+6.576,gnora


In [ ]:
for _, row in df.sample(5).iterrows():
    print(f"true label: {row['target']}")
    display(eli5.show_prediction(clf2, row['comment_text'], vec=vec2,))
    print("--"*50)

true label: 0.0


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


----------------------------------------------------------------------------------------------------
true label: 0.0


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


----------------------------------------------------------------------------------------------------
true label: 0.0


...

----------------------------------------------------------------------------------------------------
true label: 0.0


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


----------------------------------------------------------------------------------------------------
true label: 0.0


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


----------------------------------------------------------------------------------------------------


In [ ]:
y_pred= pipe.predict(df_test)

In [ ]:
y_pred = np.where(y_pred>=0.5,1.,0.)
sub_df = pd.read_csv('drive/My Drive/Jigsaw/sample_submission.csv')
sub_df['prediction'] = y_pred
sub_df.to_csv('submission.csv',index = False)

In [ ]:
y_pred_2= pipe_tfidf.predict(df_test)

In [ ]:
y_pred_2 = np.where(y_pred_2>=0.5,1.,0.)
sub_df = pd.read_csv('drive/My Drive/Jigsaw/sample_submission.csv')
sub_df['prediction'] = y_pred_2
sub_df.to_csv('submission_2.csv',index = False)

###Word embeddings and LSTM

In [5]:
from imblearn.under_sampling import RandomUnderSampler
X = df['comment_text']
X = X.values.reshape(-1,1)
y = df['target']
y = np.where(y>=0.5,1.,0.)
undersample = RandomUnderSampler(sampling_strategy='majority')
X_sample, y_sample = undersample.fit_resample(X, y)

In [6]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X_sample.reshape(-1), y_sample, 
                                                    test_size=0.3, random_state=42)

In [7]:
from keras.preprocessing.text import text_to_word_sequence
from keras.preprocessing.text import Tokenizer  
from keras.preprocessing.sequence import pad_sequences
vocab_sz = 10000
maxlen=311
tok = Tokenizer(num_words=vocab_sz, oov_token='UNK')
tok.fit_on_texts(X_sample.reshape(-1))
x_train = tok.texts_to_sequences(x_train)
x_test = tok.texts_to_sequences(x_test )
x_train = pad_sequences(x_train, maxlen=maxlen)
x_test = pad_sequences(x_test, maxlen=maxlen)

In [8]:
from keras.models import Sequential
from keras.layers import LSTM, Bidirectional, GRU, Dropout,Embedding ,Dense
model = Sequential()
model.add(Embedding(vocab_sz+1, 50, input_length=maxlen))
model.add(LSTM(64))
model.add(Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 311, 50)           500050    
                                                                 
 lstm (LSTM)                 (None, 64)                29440     
                                                                 
 dense (Dense)               (None, 1)                 65        
                                                                 
Total params: 529,555
Trainable params: 529,555
Non-trainable params: 0
_________________________________________________________________


In [9]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.fit(x_train, y_train,batch_size=64, epochs=3,validation_data=(x_test, y_test))

Epoch 1/3
3158/3158 [==============================] - 878s 277ms/step - loss: 0.3565 - accuracy: 0.8460 - val_loss: 0.3187 - val_accuracy: 0.8730
Epoch 2/3
3158/3158 [==============================] - 904s 286ms/step - loss: 0.2871 - accuracy: 0.8813 - val_loss: 0.2996 - val_accuracy: 0.8749
Epoch 3/3
3158/3158 [==============================] - 865s 274ms/step - loss: 0.2676 - accuracy: 0.8887 - val_loss: 0.2952 - val_accuracy: 0.8779


In [10]:
from sklearn import metrics

def print_report(pipe, x_test, y_test):
    y_pred = pipe.predict(x_test)
    report = metrics.classification_report(y_test, y_pred)
    print(report)
    print("accuracy: {:0.3f}".format(metrics.accuracy_score(y_test, y_pred)))

print_report(model, x_test, y_test)

ValueError: ignored

In [11]:
y_pred = model.predict(x_test)

In [16]:
from sklearn.metrics import accuracy_score
A = accuracy_score(y_test, y_pred)

print(A)

ValueError: ignored

In [ ]:
from tensorflow.keras.metrics import Precision, Recall, CategoricalAccuracy

pre = Precision()
re = Recall()
acc = CategoricalAccuracy()
for batch in test.as_numpy_iterator(): 
    # Unpack the batch 
    X_true, y_true = batch
    # Make a prediction 
    yhat = model.predict(X_true)
    
    # Flatten the predictions
    y_true = y_true.flatten()
    yhat = yhat.flatten()
    
    pre.update_state(y_true, yhat)
    re.update_state(y_true, yhat)
    acc.update_state(y_true, yhat)

print(f'Precision: {pre.result().numpy()}, Recall:{re.result().numpy()}, Accuracy:{acc.result().numpy()}')

In [ ]:
model.evaluate(x_test, y_test)

2707/2707 [==============================] - 110s 41ms/step - loss: 0.2944 - accuracy: 0.8773


[0.2943958342075348, 0.8772762417793274]

In [ ]:
test_df= pd.read_csv('drive/My Drive/Jigsaw/test.csv')
test_df= test_df['comment_text']
test_df.head()

0    [ Integrity means that you pay your debts.]\n\...
1    This is malfeasance by the Administrator and t...
2    @Rmiller101 - Spoken like a true elitist. But ...
3    Paul: Thank you for your kind words.  I do, in...
4    Sorry you missed high school. Eisenhower sent ...
Name: comment_text, dtype: object

In [ ]:
xt = tok.texts_to_sequences(test_df)
xt = pad_sequences(xt, maxlen=maxlen)

In [ ]:
y_pred= model.predict(xt)
y_pred = np.where(y_pred>=0.5,1.,0.)
sub_df = pd.read_csv('drive/My Drive/Jigsaw/sample_submission.csv')
sub_df['prediction'] = y_pred
sub_df.to_csv('submission.csv',index = False)

#GRU

In [ ]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM, SimpleRNN, GRU
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dropout, SpatialDropout1D
from tensorflow.keras.preprocessing.text import Tokenizer

In [ ]:
# The maximum number of words to be used. (most frequent)
MAX_NB_WORDS = 50000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 250
EMBEDDING_DIM = 100


In [ ]:
tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~')
tokenizer.fit_on_texts(df['comment_text'])
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 522267 unique tokens.


In [ ]:
X_final = tokenizer.texts_to_sequences(df['comment_text'].values)
X_final = pad_sequences(X_final, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X_final.shape)

Shape of data tensor: (1804874, 250)


In [ ]:
Y_final = pd.get_dummies(df , columns = ["target"] , sparse = True).values
print('Shape of label tensor:', Y_final.shape)

Shape of label tensor: (1804874, 3)


In [ ]:
#Y_final = pd.get_dummies(df['comment_text']).values
#print('Shape of label tensor:', Y_final.shape)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X_final, Y_final, test_size = 0.3, random_state = 42, shuffle=True)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(1263411, 250) (1263411, 3)
(541463, 250) (541463, 3)


In [ ]:
X_train[:5]

array([[    0,     0,     0, ...,   486,   446,  2805],
       [    0,     0,     0, ...,   410,    20,   868],
       [    0,     0,     0, ...,   283,   645,  3636],
       [    0,     0,     0, ..., 16264,    16,    58],
       [    0,     0,     0, ...,   194,     9,    56]], dtype=int32)

In [ ]:
Y_train[:5]

array([['he is smart and has good advisors. But for the rest of us middle class losers..........',
        0, 1],
       ['How about we insert Congress and all govt. employees will participate in any plan they create.',
        1, 0],
       ['Looks like one claim is debunked. http://www.subjectpolitics.com/instant-justice-trump-sexual-assault-story-immediately-debunked-3-simple-photos/',
        1, 0],
       ['Spare us the hyperbole and the inaccuracy. She wasn\'t "offering" she just reached over and did it. And yes I think most people would not appreciate someone just grabbing their hand and forcing a handshake on them.',
        1, 0],
       ["@Dai\n\nWe are now facing all the ramifications from the previous administration's actions or lack ot it, Dai. The disrespecting of our military and police, the refusal to adhere to federal laws by local government, the destructive riots caused by the left, and so on. Racial divide is so much more heightened and he was the first black presid

In [ ]:
X_train.dtype

dtype('int32')

In [ ]:
Y_train.dtype

dtype('O')

In [ ]:
gru_model = Sequential()
gru_model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X_final.shape[1]))
gru_model.add(SpatialDropout1D(0.2))
gru_model.add(GRU(128, dropout=0.2, recurrent_dropout=0.2))
gru_model.add(Dense(3, activation='softmax'))
gru_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

epochs = 5
batch_size = 64

gru_history = gru_model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.2)

ValueError: ignored

In [ ]:
gru_accuracy = gru_model.evaluate(X_test,Y_test)
print('GRU Model Evaluation\n Loss: {:0.3f}\n Accuracy: {:0.3f}'.format(gru_accuracy[0],gru_accuracy[1]))